In [2]:
from code.Matching import fuzzCompany2
import os
from code import Text_reader as tr
from code.Labels import label
from code.Text_reader import htmlDoc
import copy 
import re
from code.dataExtract import data
import code.dataExtract as de
import numpy as np

d = data(folder='../Newdata/train/')

fcomp = fuzzCompany2.from_file('fc_data2.pkl')


syn=[(),
    ('aktiengesellschaft','ag'),
    ('Limited liability company','llc'),('Public limited company','plc'),('incorporation','inc'),
    ('Sociedad por Acciones','sa'),('Public Limited','ltd'),('int','international'), ('london','bishopsgate (110)'),
    ('paris','fort dup 11046'),('london','great winche'),('london','golden lane'),('new world development','nwd'),
    ('england ','bishopsgate (110)'),
    ('london','bishopsgate (110)'),
    ('acting through its london branch','great winche'),
    ('acting through its london branch','golden lane'),
    ('incorporated with limited liability in england and wales','bishopsgate (110)'),
    ('rubbermaid inc','brands inc'),
    ('dexia funding netherlands','belfius financing company sa'),
    ('Province of British Columbia','government of the province of british columbia')]

In [3]:
def get_Guarantor(txt):
    txtc = re.sub("[^a-zA-Z0-9\s]", 
                      "",                   
                      txt)
    txtcn=txtc.replace('\n',' ').replace('\r','').lower()
    v=[-1]
    tmp=[]
    
    for item in syn:
        if len(item)==0:
            txtc = txtcn.split()
        else:
            txtc = txtcn.replace(item[0],item[1]).split()
        #print txtc
        #print '================================'
        for x in xrange(0,len(txtc)):
            if txtc[x] in fcomp.Lookup:
                tmp.append(fcomp.Lookup[txtc[x]])
            else:
                if len(tmp) > 0:
                    v.append(tmp)
                    tmp=[]
                if isinstance(v[-1],int):
                    v[-1] -= 1
                else:
                    v.append(-1)

            #if txtc[x] in ['guarantor']:  
            #    if not isinstance(v[-1],int):
            #        v.append(0)
            #    v.append('-')

            if txtc[x] in ['guarantor']:
                v.append('+')
                v.append(0)

            if len(txtc)> x+1 and txtc[x] == 'linked' and txtc[x+1] == 'to':
                v.append('*')

            if len(txtc)> x+1 and txtc[x] == 'share' and txtc[x+1] == 'issuer':
                v.append('*')
            
            if len(txtc)> x+1 and txtc[x] == 'equity' and txtc[x+1] == 'issuer':
                v.append('*')
            
            if txtc[x] == 'issuer':
                v.append('*')
            

    group=set()
    x=0
    while x<len(v):
        if v[x]=='*' and x+2 < len(v):
            x+=2
            continue
        if v[x]=='-' and x > 2:
            group.add((abs(v[x-1]),tuple(v[x-2])))
        if v[x]=='+' and x+2 < len(v):
            group.add((abs(v[x+1]),tuple(v[x+2]))) 
        x+=1

    r=[]

    x=0
    for item in group:
        subset = fcomp.get_subsets(item[1])
        ps = fcomp.get_Ps_vect(subset)
        idx = np.argmax(ps)
        #print x,fcomp.values[idx], ps[idx]/float(item[0]+1)*len(fcomp.values[idx].split()),item[0]
        x+=1
        r.append((ps[idx]/float(item[0]+1)*len(fcomp.values[idx].split()), fcomp.values[idx]))

    r = sorted(r,key=lambda x:x[0],reverse=True)
    if len(r)>0:
        return r[0][1]
    else:
        return None


In [4]:
folder='../Newdata/train/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../Newdata/train/txt')

GuarantorDic={}

x=0
for isin in d.docid: # for each isin in the data object created before
    
    if x > 200:
        break
    x+=1
    
    f=d.docid[isin] # get the file_ids
    Guarantor=None
    for item in f: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        Guarantor=get_Guarantor(txt)
        if Guarantor is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            Guarantor=get_Guarantor(txt)
            
        if Guarantor is not None:
            break
    
    print isin, Guarantor
    if Guarantor is not None:
        GuarantorDic[isin]=Guarantor
    else:
        GuarantorDic[isin]=''

XS1378463068 None
XS1267246327 None
XS1314875508 THE KROGER CO
US50066RAB24 None
XS1398577475 None
XS1346593194 None
XS1343808702 None
XS1254344952 None
CH0295931650 EXANE DERIVATIVES
XS1326554562 None
XS1333969266 None
XS1402911298 None
XS1354249945 THE KROGER CO
CH0287822685 EFG INTERNATIONAL AG
XS1313801786 BNP PARIBAS
XS1305149954 THE KROGER CO
XS1274660338 None
XS1337149105 THE KROGER CO
XS1358065362 None
XS0536851537 CITIBANK NA
XS1267285572 None
XS1378444944 None
CH0279927146 EFG INTERNATIONAL AG
XS1378425059 None
XS1385960874 None
XS1289327550 None
XS1267269378 None
XS1328227191 None
XS0586136573 FLUXYS BELGIUM SA
XS1326563472 None
XS1340127908 None
CH0242059902 KOMMUNALBANKEN AS
XS1326566814 None
XS1402188129 BRITISH AND MALAYAN TRUSTEES LIMITED
USU88803AE84 21ST CENTURY FOX AMERICA, INC
CH0315411808 None
XS1308051918 None
XS1293137573 BNP PARIBAS
XS1319996747 None
CH0254071431 TERNA RETE ELETTRICA NAZIONALE SOCIETA' PER AZIONI (IN FORMA ABBREVIATO ""TERNA S.P.A."")
XS13244000

In [5]:
guarantorAns = {}
x=0
for item in d.guarantor:
    if item in GuarantorDic:  
        if len(d.guarantor[item]) > 0:
            guarantorAns[item] = d.guarantor[item][0]
        else:
            guarantorAns[item] = ''
    
de.test(GuarantorDic, guarantorAns)


total number of set compared: 201.0
total number of missing set: 0.0
error %: 31.8407960199


In [6]:
for item in GuarantorDic:
    #if GuarantorDic[item] != guarantorAns[item]:
        print GuarantorDic[item],', ',guarantorAns[item]

KOMMUNALBANKEN AS ,  RAIFFEISEN SCHWEIZ GENOSSENSCHAFT
 ,  
 ,  
 ,  
THE KROGER CO ,  SOCIETE GENERALE
 ,  
 ,  
 ,  
 ,  
 ,  
 ,  
 ,  
 ,  
BANK HAPOALIM BM ,  BANK HAPOALIM BM
 ,  
THE KROGER CO ,  
 ,  
 ,  
 ,  
EXANE DERIVATIVES ,  EXANE DERIVATIVES
 ,  DEUTSCHE BANK AG, AVENUE OF THE AMERICAS 1301
 ,  
 ,  
 ,  DEUTSCHE BANK AG, AVENUE OF THE AMERICAS 1301
GARFUNKELUX HOLDCO 3 S.A. ,  BOC INTERNATIONAL HOLDINGS LIMITED
TERNA RETE ELETTRICA NAZIONALE SOCIETA' PER AZIONI (IN FORMA ABBREVIATO ""TERNA S.P.A."") ,  
 ,  
 ,  
 ,  
 ,  
BANCO SANTANDER SA ,  BANCO SANTANDER SA
THE KROGER CO ,  
 ,  SONDERFONDS FINANZMARKTSTABILISIERUNG (SOFFIN)
 ,  
EFG INTERNATIONAL AG ,  EFG INTERNATIONAL FINANCE (GUERNSEY) LIMITED
 ,  
BNP PARIBAS ,  BNP PARIBAS SA
THE KROGER CO ,  EFG-HERMES HOLDING COMPANY SAE
THE KROGER CO ,  
 ,  
 ,  
THE KROGER CO ,  SOCIETE GENERALE
 ,  
 ,  
KINGDOM OF THE NETHERLANDS ,  MOLSON COORS BREWING COMPANY
THE KROGER CO ,  SOCIETE GENERALE
BANCO BILBAO VIZCAYA A